In [6]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import *
from spytial.annotations import *


# Disjoint Sets (Union - Find)

In [16]:
# CLRS Union–Find (Disjoint Sets) — element-pointer API
# MAKE-SET, FIND-SET (path compression), UNION (by rank)

@attribute(field="key")
@attribute(field="rank")
@orientation(selector='{ x, y : DSUNode | (x != y) and (x.parent = y) }',
             directions=['below'])   # parent above, children below
#@hideField(selector='{ x : DSUNode | x.parent = x}', field='parent')
class DSUNode:
    def __init__(self, key):
        self.key = key
        self.parent = self   # MAKE-SET
        self.rank = 0        # tree rank (upper bound on height)

@flag(name='hideDisconnected')
class DisjointSet:
    def make_set(self, key):
        return DSUNode(key)

    # FIND-SET with path compression
    def find_set(self, x: DSUNode) -> DSUNode:
        if x.parent is not x:
            x.parent = self.find_set(x.parent)
        return x.parent

    # UNION by rank
    def union(self, x: DSUNode, y: DSUNode) -> DSUNode:
        return self._link(self.find_set(x), self.find_set(y))

    def _link(self, x_root: DSUNode, y_root: DSUNode) -> DSUNode:
        if x_root is y_root:
            return x_root
        if x_root.rank > y_root.rank:
            y_root.parent = x_root
            return x_root
        else:
            x_root.parent = y_root
            if x_root.rank == y_root.rank:
                y_root.rank += 1
            return y_root


# TODO: 

THIS IS NOT WORKING!!

In [18]:
# Create a DisjointSet instance and some DSU nodes, perform unions, and render a diagram.
ds = DisjointSet()

# MAKE-SET for elements
a = ds.make_set("A")
b = ds.make_set("B")
c = ds.make_set("C")
d = ds.make_set("D")
e = ds.make_set("E")  # will remain a separate set



# Perform unions to form some trees:
z = ds.union(a, b)  # union A and B

# Why is this NOT showing?
diagram(z)

ds.union(c, d)   # union C and D
ds.union(b, c)   # connect the two trees (A,B) and (C,D) -> one larger set

# Run finds to apply path compression (so parent links are updated)
for node in (a, b, c, d, e):
    ds.find_set(node)


